# Training Ingredient Parser
This is the test area for training a parser.
[Reference Material](https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb)

In [2]:
# matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [4]:
from itertools import chain

import csv

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [19]:
entries = []
headers = []
first = 1

with open('nyt-ingredients-snapshot-2015.csv') as csvfile:
    ingredient_reader = csv.reader(csvfile)
    for row in ingredient_reader:
        if first:
            headers += row
            first = 0
        else:
            entries.append(row)

> Note that this is currently only trying to get the correct quantity/amount of the ingredient

In [42]:
def sent2features(sent):
    nonAlpha = ""
    try:
        words = sent[1].split(' ')
        word = words.pop(0)
        while(not word.isalpha()):
            nonAlpha += word + " "
            word = words.pop(0)
    except:
        pass
    nonAlpha = nonAlpha.strip()
    return [{'sent': sent[1], 'nonAlpha': nonAlpha}]

def sent2labels(sent):
    return [sent[3]]

In [43]:
X_train = [sent2features(s) for s in entries[:50000]]
y_train = [sent2labels(s) for s in entries[:50000]]

X_test = [sent2features(s) for s in entries[51000:100000]]
y_test = [sent2labels(s) for s in entries[51000:100000]]

In [44]:
X_train[:10]

[[{'sent': '1 1/4 cups cooked and pureed fresh butternut squash, or 1 10-ounce package frozen squash, defrosted',
   'nonAlpha': '1 1/4'}],
 [{'sent': '1 cup peeled and cooked fresh chestnuts (about 20), or 1 cup canned, unsweetened chestnuts',
   'nonAlpha': '1'}],
 [{'sent': '1 medium-size onion, peeled and chopped',
   'nonAlpha': '1 medium-size onion,'}],
 [{'sent': '2 stalks celery, chopped coarse', 'nonAlpha': '2'}],
 [{'sent': '1 1/2 tablespoons vegetable oil', 'nonAlpha': '1 1/2'}],
 [{'sent': '', 'nonAlpha': ''}],
 [{'sent': '2 tablespoons unflavored gelatin, dissolved in 1/2 cup water',
   'nonAlpha': '2'}],
 [{'sent': 'Salt', 'nonAlpha': ''}],
 [{'sent': '1 cup canned plum tomatoes with juice', 'nonAlpha': '1'}],
 [{'sent': '6 cups veal or beef stock', 'nonAlpha': '6'}]]

In [46]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 2min 5s, sys: 31.2 ms, total: 2min 5s
Wall time: 2min 5s


/home/aaron/.local/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [50]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=list(crf.classes_))

/home/aaron/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9663775050525185

TODO -> run this to try to get the unit and then the ingredient itself